# **Latihan: Deploy Model ML Menggunakan TensorFlow Serving**
Pada materi sebelumnya kita telah berkenalan dengan TF-Serving. Selain itu, kita juga sudah menyiapkan sistem TF-Serving menggunakan Docker image. Nah, pada materi ini kita akan belajar proses deployment model menggunakan TF-Serving. 

Berikut beberapa hal yang akan kita pelajari pada latihan kali ini:

Membuat dan menjalankan model menggunakan TF-Serving.
Menguji hasil deployment dengan membuat request prediksi menggunakan API yang ada pada TF-Serving.
Anda dapat mengunduh contoh proyek pada tautan berikut: contoh proyek. 

Tugas Anda adalah melengkapi bagian yang kosong sesuai instruksi yang diberikan.



# **Kebutuhan Sistem**
Anda akan menjalankan model pada server menggunakan TF-Serving. Sebelum itu, tentunya Anda perlu menyiapkan beberapa kebutuhan sistem, antara lain:

1. Docker
2. Docker image untuk TF-Serving
3. TF-Serving Python API

# **Penjelasan Proyek**
Tujuan dari proyek ini adalah membuat sebuah model yang dapat melakukan klasifikasi gambar rock-paper-scissors. Model ini nantinya akan dimuat dan dijalankan menggunakan TF-Serving https://www.tensorflow.org/tfx/guide/serving.

Dataset yang akan digunakan adalah rock-paper-scissors. Dataset tersebut memiliki tiga kategori, tiap kategorinya memiliki lebih dari 700 sampel gambar. Berikut beberapa sampel gambar dari setiap kategori.

# **Penjelasan File**
Setelah mengunduh contoh proyek, Anda akan menjumpai beberapa file dan folder sebagai berikut:

* `dataset` merupakan sebuah folder berisi dataset untuk melatih dan mengevaluasi model.
* ‘notebook.ipynb’ merupakan sebuah berkas notebook yang dapat dijalankan melalui Google Colab. Notebook ini berisi tahapan dalam membuat dan menyimpan model dalam format SavedModel. 
* ‘models’ merupakan sebuah folder yang berisi model yang disimpan dalam format SavedModel.
* ‘image’ merupakan sebuah folder yang berisi beberapa gambar yang akan digunakan sebagai input gambar untuk menguji hasil deployment.
* ‘Test.ipynb’ merupakan sebuah berkas notebook yang digunakan untuk membuat request prediksi menggunakan API yang ada pada TF-Serving.

Berikut merupakan tree structure dari contoh proyek ini:

In [ ]:
├───dataset
│   ├───testing
│   │   ├───paper
│   │   ├───rock
│   │   └───scissors
│   └───training
│       ├───paper
│       ├───rock
│       └───scissors
├───images
├───models
    └───rps_model
        └───1
            ├───assets
            └───variables

# **Penjelasan Notebook**
Bagian ini akan membahas tahap pembuatan model untuk mengklasifikasi gambar rock-paper-scissors. Sebelum membuat model, siapkan semua library yang dibutuhkan.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import os

Tahap selanjutnya adalah mempersiapkan dataset yang akan digunakan. Kemudian, lakukan tahap preprocessing data berikut:

* Melakukan image augmentation
* Mempersiapkan data latih dan data evaluasi

In [ ]:
training_dir = 'dataset/training/'
validation_dir = 'dataset/testing/'
 
 
training_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
 
training_generator = training_datagen.flow_from_directory(training_dir,
                                                     batch_size=126,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))
 
validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
 
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                                batch_size=128,
                                                                class_mode='categorical',
                                                                target_size=(150, 150))

Berikutnya, buat dan latihlah model menggunakan dataset yang telah kita siapkan sebelumnya. Proyek ini menggunakan teknik transfer learning memanfaatkan model `MobileNetV2`.

In [ ]:
pre_trained_model = MobileNetV2(weights="imagenet", include_top=False,
                                input_tensor=Input(shape=(150, 150, 3)))
 
for layer in pre_trained_model.layers:
    layer.trainable = False
 
last_output = pre_trained_model.output
 
 
x = tf.keras.layers.Flatten(name="flatten")(last_output)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)
 
model = tf.keras.models.Model(pre_trained_model.input, x)
 
 
int_lr = 1e-4
num_epochs = 10
 
optimizer = tf.optimizers.Adam(lr=int_lr)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
 
H = model.fit(training_generator,
              epochs=num_epochs,
              validation_data=validation_generator)

Jika performa model dirasa cukup, kita dapat menyimpan model tersebut dalam format SavedModel menggunakan perintah berikut:

In [ ]:
save_path = os.path.join("models/rps_model/1/")
tf.saved_model.save(model, save_path)

Pada tahap ini, Anda telah berhasil membuat model untuk mengklasifikasi gambar rock-paper-scissors dan menyimpannya dalam format SavedModel. Pada tahap berikutnya, Anda akan menjalankan model menggunakan TF-Serving. 



# **Proses Deployment Model Menggunakan TF-Serving**
Sebelum menerapkan model dengan TF-Serving, pastikan kembali Anda telah menginstal Docker image untuk TF-Serving dan TF-Serving Python API. 

Selanjutnya, Anda dapat mulai proses deployment dengan menjalankan perintah berikut pada PowerShell/Terminal:

In [ ]:
docker run -it -v YOUR_PATH\models:/models -p 8501:8501 --entrypoint /bin/bash tensorflow/serving

Perintah di atas akan dieksekusi oleh Docker container. Apakah Anda masih bingung? Simak penjelasan setiap perintahnya di bawah ini:

* Bagian ‘--it’ merupakan perintah untuk membuat Docker container menjadi interaktif. 
* Bagian ‘-v’ berfungsi untuk mengarahkan direktori lokal  (YOUR_PATH\models) ke dalam directory container (/models). 
* Bagian ‘-p 8501:8501’ berfungsi untuk meneruskan TCP port host (8501) ke TCP port container (8501).
* Bagian ‘--entrypoint /bin/bash’ digunakan untuk masuk ke bagian shell Docker container.

Gambar di bawah ini merupakan contoh tampilan shell Docker container pada PowerShell/Terminal.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:3062d732af5e8a785bc001f08bccfddf20211214101720.jpeg

Tahap selanjutnya adalah menjalankan model pada Docker container dengan perintah berikut:

In [ ]:
tensorflow_model_server --rest_api_port=8501 --model_name=rps_model --model_base_path=/models/rps_model/

Jika Anda berhasil menjalankan model pada Docker container, tampilan PowerShell/terminal akan terlihat seperti di bawah ini.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:e1997370a0bec271a8a5bfdf5b8173d120211209090239.jpeg

Setelah menjalankan model pada Docker container, ujilah hasil deployment dengan menyalin URL (http://localhost:8501/v1/models/rps_model) pada alamat web browser. Jika proses deployment berjalan dengan baik, akan muncul respon JSON sebagai berikut:

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:11d9e79b6b2d2e0cfe380fc97a8a7dda20211209090241.jpeg

Hasil di atas merupakan status dari model yang dijalankan melalui TF-Serving. Hasil ini menunjukkan bahwa kita memiliki sebuah model versi 1 yang berstatus “AVAILABLE”.



# **Menguji Hasil Deployment**
Pada tahap ini, Anda akan menguji hasil deployment dengan membuat request prediksi untuk mengklasifikasi gambar rock-paper-scissors. Request prediksi ini dijalankan dengan API yang ada pada TF-Serving.

Perlu diingat, input gambar yang Anda masukkan tidak bisa langsung diolah oleh model. Sehingga, perlu dilakukan tahap preprocessing terlebih dahulu. Pada tahap preprocessing, ada beberapa hal yang bisa dilakukan, antara lain:

* Mengubah format gambar menjadi image_tensor.
* Mengubah ukuran gambar.
* Melakukan normalisasi.

In [ ]:
def images_preprocessing(filename):
   
    image = tf.io.decode_image(open(filename, 'rb').read(), channels=3)
    image = tf.image.resize(image, [150, 150])
    image = image/255.
   
    image_tensor = tf.expand_dims(image, 0)
    image_tensor = image_tensor.numpy().tolist()
   
    return image_tensor
 
filename = 'images\Batu_b.jpg'
image_tensor = images_preprocessing(filename=filename)

Selanjutnya, kita akan mengubah hasil preprocessing ke bentuk JSON. JSON ini digunakan sebagai masukan ketika membuat request prediksi.

In [ ]:
json_data = {
    "instances": image_tensor
}

Sebelum meminta request prediksi, definisikan terlebih dahulu API endpoint yang ingin digunakan. Selanjutnya buatlah sebuah request untuk melakukan prediksi menggunakan endpoint tersebut. 

Request prediksi akan menghasilkan output berupa nilai probabilitas sebuah gambar yang diklasifikasikan ke dalam kategori atau label tertentu. Sehingga, kita perlu mengambil indeks label dengan nilai probabilitas paling tinggi menggunakan fungsi tf.argmax. Indeks ini selanjutnya akan dipetakan ke dalam kategori yang sesuai.

In [ ]:
endpoint = "http://localhost:8501/v1/models/rps_model:predict"
 
response = requests.post(endpoint, json=json_data)
 
prediction = tf.argmax(response.json()['predictions'][0]).numpy()
 
map_labels = {0: "Paper", 1: "Rock", 2: "Scissors"}
print(map_labels[prediction])

Jika semua tahapan berjalan dengan baik, potongan kode di atas akan  menghasilkan output sebagai berikut:

In [ ]:
Rock

Selamat!. Anda telah berhasil melakukan proses deployment model machine learning menggunakan TF-Serving